In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import ollama  
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [2]:
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5
)

In [3]:
# Example JSON response template
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [4]:
# Template for generating MCQs using the provided text and guidelines
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to conform to the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}.
"""

In [5]:
# PromptTemplate for generating MCQs
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [6]:
# Chain for generating the quiz
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


C:\Users\shubh\AppData\Local\Temp\ipykernel_15792\23162514.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [7]:
# Template for evaluating the quiz generated by the model
TEMPLATE2 = """
You are an expert English grammarian and writer with domain knowledge of farming field. Given a Multiple Choice Quiz for {subject} students,\
you need to evaluate the complexity of the question and provide a complete analysis of the quiz. Use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed, and adjust the tone to perfectly fit student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English writer the above quiz:
"""

In [8]:
# PromptTemplate for evaluating and refining the quiz
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

# Chain for evaluating the quiz
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [9]:
# SequentialChain to handle quiz generation and evaluation
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [10]:
# File path to the text file containing the content to generate MCQs
file_path = r"D:\MCQ_Generator\mush.txt"

# Read the content from the file
with open(file_path, 'r') as file:
    TEXT = file.read()

TEXT

'1. Could protect against dementia\nThe brain’s ability to grow and form new connections typically declines with age, which may explain why mental functioning worsens in many older adults (2Trusted Source).\n\nStudies have found that lion’s mane mushrooms contain two special compounds that can stimulate the growth of brain cells: hericenones and erinacines (3Trusted Source).\n\nAdditionally, animal studies have found that lion’s mane may help protect against Alzheimer’s disease, a degenerative brain disease that causes progressive memory loss.\n\nIn fact, lion’s mane mushroom and its extracts have been shown to reduce symptoms of memory loss in mice, as well as prevent neuronal damage caused by amyloid-beta plaques, which accumulate in the brain during Alzheimer’s disease (4Trusted Source, 5Trusted Source, 6Trusted Source).\n\nA 2020 study of people with mild Alzheimer’s disease found that supplementation with 1 gram of lion’s mane mushroom daily for 49 weeks significantly improved cog

In [11]:
# Call the generate_evaluate_chain to generate and evaluate the MCQs
with get_openai_callback() as cb:
    response = generate_evaluate_chain({
        "text": TEXT,
        "number": 9,  # Example: Generate 5 MCQs
        "subject": "Lion's Mane Mushrooms",
        "tone": "Normal",
        "response_json": RESPONSE_JSON
    })

C:\Users\shubh\AppData\Local\Temp\ipykernel_15792\3530025741.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain({




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:1. Could protect against dementia
The brain’s ability to grow and form new connections typically declines with age, which may explain why mental functioning worsens in many older adults (2Trusted Source).

Studies have found that lion’s mane mushrooms contain two special compounds that can stimulate the growth of brain cells: hericenones and erinacines (3Trusted Source).

Additionally, animal studies have found that lion’s mane may help protect against Alzheimer’s disease, a degenerative brain disease that causes progressive memory loss.

In fact, lion’s mane mushroom and its extracts have been shown to reduce symptoms of memory loss in mice, as well as prevent neuronal damage caused by amyloid-beta plaques, which accumulate in the brain during Alzheimer’s disease (4Trusted Source, 5Trusted Source, 6Trusted Source).

A 2020 study of people with mild Alzheimer’s disease found that

In [12]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3364
Prompt Tokens:1821
Completion Tokens:1543
Total Cost:0.0


In [13]:
response

{'text': '1. Could protect against dementia\nThe brain’s ability to grow and form new connections typically declines with age, which may explain why mental functioning worsens in many older adults (2Trusted Source).\n\nStudies have found that lion’s mane mushrooms contain two special compounds that can stimulate the growth of brain cells: hericenones and erinacines (3Trusted Source).\n\nAdditionally, animal studies have found that lion’s mane may help protect against Alzheimer’s disease, a degenerative brain disease that causes progressive memory loss.\n\nIn fact, lion’s mane mushroom and its extracts have been shown to reduce symptoms of memory loss in mice, as well as prevent neuronal damage caused by amyloid-beta plaques, which accumulate in the brain during Alzheimer’s disease (4Trusted Source, 5Trusted Source, 6Trusted Source).\n\nA 2020 study of people with mild Alzheimer’s disease found that supplementation with 1 gram of lion’s mane mushroom daily for 49 weeks significantly imp

In [14]:
quiz=response.get("quiz")

In [15]:
quiz

"{'1': {'mcq': 'What are some potential health benefits of Lion\\'s Mane mushrooms?', 'options': {'a': 'Improved cognitive function, reduced inflammation, and boosted immune system', 'b': 'Increased risk of disease, liver damage, and stroke', 'c': 'Reduced muscle mass and bone density', 'd': 'Decreased energy levels'}, 'correct': 'a'}, \n'2': {'mcq': 'How can Lion\\'s Mane mushrooms help with obesity?', 'options': {'a': 'By increasing inflammation in fat tissue', 'b': 'By decreasing the amount of inflammation fat tissue releases', 'c': 'By reducing appetite', 'd': 'By increasing metabolism'}, 'correct': 'b'}, \n'3': {'mcq': 'What is one potential benefit of Lion\\'s Mane mushrooms for people with mild Alzheimer\\'s disease symptoms?', 'options': {'a': 'Improved cognitive function', 'b': 'Reduced risk of disease', 'c': 'Increased energy levels', 'd': 'Improved sleep quality'}, 'correct': 'a'}, \n'4': {'mcq': 'How can Lion\\'s Mane mushrooms be prepared for consumption?', 'options': {'a'

In [18]:
quiz = quiz.replace("'", '"').replace("\\'", "'")

# Print cleaned quiz for debugging
print(quiz)


{"1": {"mcq": "What are some potential health benefits of Lion\"s Mane mushrooms?", "options": {"a": "Improved cognitive function, reduced inflammation, and boosted immune system", "b": "Increased risk of disease, liver damage, and stroke", "c": "Reduced muscle mass and bone density", "d": "Decreased energy levels"}, "correct": "a"}, 
"2": {"mcq": "How can Lion\"s Mane mushrooms help with obesity?", "options": {"a": "By increasing inflammation in fat tissue", "b": "By decreasing the amount of inflammation fat tissue releases", "c": "By reducing appetite", "d": "By increasing metabolism"}, "correct": "b"}, 
"3": {"mcq": "What is one potential benefit of Lion\"s Mane mushrooms for people with mild Alzheimer\"s disease symptoms?", "options": {"a": "Improved cognitive function", "b": "Reduced risk of disease", "c": "Increased energy levels", "d": "Improved sleep quality"}, "correct": "a"}, 
"4": {"mcq": "How can Lion\"s Mane mushrooms be prepared for consumption?", "options": {"a": "Only c

In [19]:
quiz=json.loads(quiz)

In [20]:
quiz

{'1': {'mcq': 'What are some potential health benefits of Lion"s Mane mushrooms?',
  'options': {'a': 'Improved cognitive function, reduced inflammation, and boosted immune system',
   'b': 'Increased risk of disease, liver damage, and stroke',
   'c': 'Reduced muscle mass and bone density',
   'd': 'Decreased energy levels'},
  'correct': 'a'},
 '2': {'mcq': 'How can Lion"s Mane mushrooms help with obesity?',
  'options': {'a': 'By increasing inflammation in fat tissue',
   'b': 'By decreasing the amount of inflammation fat tissue releases',
   'c': 'By reducing appetite',
   'd': 'By increasing metabolism'},
  'correct': 'b'},
 '3': {'mcq': 'What is one potential benefit of Lion"s Mane mushrooms for people with mild Alzheimer"s disease symptoms?',
  'options': {'a': 'Improved cognitive function',
   'b': 'Reduced risk of disease',
   'c': 'Increased energy levels',
   'd': 'Improved sleep quality'},
  'correct': 'a'},
 '4': {'mcq': 'How can Lion"s Mane mushrooms be prepared for consu

In [21]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [22]:
quiz_table_data

[{'MCQ': 'What are some potential health benefits of Lion"s Mane mushrooms?',
  'Choices': 'a: Improved cognitive function, reduced inflammation, and boosted immune system | b: Increased risk of disease, liver damage, and stroke | c: Reduced muscle mass and bone density | d: Decreased energy levels',
  'Correct': 'a'},
 {'MCQ': 'How can Lion"s Mane mushrooms help with obesity?',
  'Choices': 'a: By increasing inflammation in fat tissue | b: By decreasing the amount of inflammation fat tissue releases | c: By reducing appetite | d: By increasing metabolism',
  'Correct': 'b'},
 {'MCQ': 'What is one potential benefit of Lion"s Mane mushrooms for people with mild Alzheimer"s disease symptoms?',
  'Choices': 'a: Improved cognitive function | b: Reduced risk of disease | c: Increased energy levels | d: Improved sleep quality',
  'Correct': 'a'},
 {'MCQ': 'How can Lion"s Mane mushrooms be prepared for consumption?',
  'Choices': 'a: Only cooked and dried | b: Raw, cooked, dried, or steeped a

In [23]:
quiz=pd.DataFrame(quiz_table_data)

In [24]:
quiz.to_csv("mushroom.csv",index=False)

In [25]:
quiz

,MCQ,Choices,Correct
0,What are some potential health benefits of Lio...,"a: Improved cognitive function, reduced inflam...",a
1,"How can Lion""s Mane mushrooms help with obesity?",a: By increasing inflammation in fat tissue | ...,b
2,"What is one potential benefit of Lion""s Mane m...",a: Improved cognitive function | b: Reduced ri...,a
3,"How can Lion""s Mane mushrooms be prepared for ...","a: Only cooked and dried | b: Raw, cooked, dri...",b
4,"What is a potential side effect of Lion""s Mane...",a: Difficulty breathing or skin rashes | b: In...,a
5,"What is the recommended dosage of Lion""s Mane ...",a: 1-2 grams per day | b: 3-5 grams per day | ...,b
6,"What is one potential benefit of Lion""s Mane m...",a: Improved cognitive function | b: Reduced ri...,a
7,"How can Lion""s Mane mushrooms be obtained in s...",a: Only in capsule and tablet form | b: In var...,b
8,"What is a potential risk of taking Lion""s Mane...",a: Increased risk of disease | b: Reduced effe...,c
